# Metabolomics Workbench Enzyme Metabolite Associations Croissant Builder 🥐

Author: Ido Diamant, Ma'ayan Lab, CFDE DRC

In [1]:
import datetime
import hashlib
import json
import mlcroissant as mlc
import os
import pandas as pd
import requests
from zipfile import ZipFile

## Construct Croissant from CFDE Attribute Table

In [2]:
name='MW Enzyme Metabolite Associations'
description="Biomolecular interactions between metabolites and proteins curated from experimental studies"

In [3]:
short_citation = 'Sud, Nucleic Acids Res, 2016'
title = 'Metabolomics Workbench: An international repository for metabolomics data and metadata, metabolite standards, protocols, tutorials and training, and analysis tools'
author = 'Sud, M'
journal = 'Nucleic Acids Res'
year = 2016
volume = 44
pages = 'D463-D477'

cite_as=(f'@article{{{short_citation}, title={{{title}}}, author={{{author}}}, journal={{{journal}}}, year={{{year}}}, volume={{{volume}}}, pages={{{pages}}}}}')

In [4]:
creators = [
    mlc.Organization(name="Metabolomics Workbench", url="https://www.metabolomicsworkbench.org/")
]

publishers = [
    mlc.Organization(name="Ma'ayan Lab", url="https://maayanlab.cloud/")
]

license = "https://www.metabolomicsworkbench.org/about/termsofuse.php"
version = "0.1.0"
url="https://maayanlab.cloud/Harmonizome/dataset/Metabolomics+Workbench+Enzyme+Metabolite+Associations"
date_published=datetime.date(2023, 10, 24)

### Resource

In [5]:
# Utility function to generate a SHA256 checksum for a FileObject from a URL
def get_sha256(url):
    sha256 = hashlib.sha256()
    response = requests.get(url, stream=True)
    for chunk in response.iter_content(chunk_size=65536):
        sha256.update(chunk)
    return sha256.hexdigest()

In [6]:
file_url = 'https://maayanlab.cloud/static/hdfs/harmonizome/data/mwmetabolites/gene_attribute_matrix.txt.gz'

distribution = [
    mlc.FileObject(
        id="dataset-attribute-table-archive",
        name="dataset-attribute-table-archive",
        description="Dataset attribute table archive from Harmonizome.",
        content_url=file_url,
        encoding_formats="application/gzip",
        sha256=get_sha256(file_url)
    ),
    mlc.FileObject(
        id="dataset-attribute-table",
        name="dataset-attribute-table",
        description="Dataset attribute table from Harmonizome.",
        content_url="gene_attribute_matrix.txt",
        encoding_formats="text/tab-separated-values",
        contained_in={"dataset-attribute-table-archive"}
    )
]

### Structure/Semantics

In [7]:
matrix = pd.read_csv(file_url, sep='\t', compression='gzip')
display(matrix)

,gene_symbol,"(2-trans,6-trans)-Farnesal","(24S)-Cholest-5-ene-3beta,7alpha,24-triol","(3S)-3-Hydroxy-N6,N6,N6-trimethyl-L-lysine","(4Z,7Z,10Z,13Z,16Z,19Z)-Docosahexaenoyl-CoA",(5-L-Glutamyl)-L-amino acid,(R)-3-Hydroxybutanoate,(S)-1-Piperideine-6-carboxylic acid,"(S)-2,3-Epoxysqualene",(S)-2-Aminobutanoate,...,gamma-Aminobutyric acid,gamma-Glutamylcysteine,p-Hydroxyfelbamate,p-Hydroxyphenylacetic acid,sn-Glycero-3-phosphate,sn-Glycero-3-phosphocholine,sn-Glycero-3-phosphoethanolamine,"trans,cis-Lauro-2,6-dienoyl-CoA","trans,trans-Farnesyl diphosphate",trans-2-Enoyl-OPC6-CoA
0,AADAT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AANAT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AARS1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AARS2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AASDHPPT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,VNN1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1046,WARS1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1047,WARS2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1048,YARS1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
fields = []
array_size = matrix.shape[0]

fields.append(
  mlc.Field(
    id="associations/gene",
    name="gene",
    description="The NCBI gene symbol",
    data_types=mlc.DataType.TEXT,
    is_array=True,
    array_shape=str(array_size),
    source=mlc.Source(
      file_object="dataset-attribute-table",
      extract=mlc.Extract(column="Gene")
    )
  )
)

for col in matrix.columns[1:]:
  fields.append(
    mlc.Field(
      id=f"associations/{col.replace(' ','_')}",
      name=f"associations/{col}",
      data_types=mlc.DataType.INTEGER,
      is_array=True,
      array_shape=str(array_size),
      source=mlc.Source(
        file_object="dataset-attribute-table",
        extract=mlc.Extract(column=col)
      )
    )
  )

print(len(fields))

735


In [9]:
record_sets = [
    mlc.RecordSet(
        id="associations",
        name="associations",
        key='associations/gene',
        fields = fields
    )
]

### Assemble Dataset and Write JSON
Once we have defined each dataset layer, we can package the dataset metadata into a Croissant JSON-LD file.

In [10]:
metadata = mlc.Metadata(
    name = name,
    description = description,
    cite_as = cite_as,
    url = url,
    date_published=date_published,
    creators = creators,
    publisher = publishers,
    license = license,
    version = version,
    distribution = distribution,
    record_sets = record_sets
)

# Display any warnings/suggestions encountered by the validator when building the metadata
print(metadata.issues.report())

In [11]:
with open("mwmetabolites.json", "w") as f:
  content = metadata.to_json()
  content = json.dumps(content, indent=4, default=str)
  f.write(content)
  f.write("\n")  # Terminate file with newline